# Linear Regression - Gradient Descent


In this notebook we investigate an iterative optimization method known as Gradient Descent (GD) to solve the Linear Regression problem. We will use the sklearn's Gradient Descent model.


<font color=red size=3> **Application Scenario:**</font> 
The GD approach is suitable for the following scenario.
- Dataset: can be very large large (unable to fit into computer's memory)
- No. of Features: large
- Relationship between input (features) and output (target): Linear & Non-Linear (Polynomial Regression)
- Out-of-core support: Out-of-core algorithms can handle vast quantities of data that cannot fit in a computer’s main memory. An out-of-core learning algorithm chops the data into mini-batches and uses online learning techniques to learn from these mini-batches.


## Tasks
We perform the following tasks.
- Task 1: Implement GD on a Linear Regression Model
- Task 2: Implement GD on a Polynomial Regression Model

We will see that the Polynomial Regression model has more power to discover the non-linear pattern in the dataset, however, it overfits (high-variance and less generalizable). By implementing a regularized Polynomial Regression model using GD we can obtain better generalizable solution. Note that we obtained similar optimal solution in notebook 3 by using the OLS regularized Polynimial Regression model.



## Gradient Descent
The gradient descent based methods are used when there are a large number of features, or too many training instances to fit in memory.

There are three variants of the Gradient Descent Algorithms.
- Batch Gradient Descent
- Stochastic Gradient Descent (SGD)
- Mini-batch Gradient Descent

Scikit-Learn provides only the Stochastic Gradient Descent model. In this notebook, we will see how to use sklearn's SGDRegressor.


### Batch Gradient Descent
Batch Gradient Descent uses the whole batch of training data at every step. As a result it is terribly slow on very large training sets.


### Stochastic Gradient Descent

The main problem with Batch Gradient Descent is the fact that it uses the whole training set to compute the gradients at every step, which makes it very slow when the training set is large. At the opposite extreme, Stochastic Gradient Descent (SGD) just picks a random instance in the training set at every step and computes the gradients based only on that single instance. Obviously this makes the algorithm much faster since it has very little data to manipulate at every iteration. It also makes it possible to train on huge training sets, since only one instance needs to be in memory at each iteration.

On the other hand, due to its stochastic (i.e., random) nature, this algorithm is much less regular than Batch Gradient Descent. Instead of gently decreasing until it reaches the minimum, the cost function will bounce up and down, decreasing only on average. Over time it will end up very close to the minimum, but once it gets there it will continue to bounce around, never settling down. So once the algorithm stops, the final parameter values are good, but not optimal.


## Implementation Issues:

We need to consider the following two issues for implementing gradient descent algorithms.

####  Learning Rate
When implementing the Gradient Descent algorithm, if the learning rate is set to a too low value, the algorithm will eventually reach the solution, but it will take a long time. On the other hand, if the learning rate is too high, the algorithm diverges, jumping all over the place and actually getting further and further away from the solution at every step.

To find a good learning rate, we can use grid search. However, we may want to limit the number of iterations so that grid search can eliminate models that take too long to converge.


#### No. of Iterations
Setting the number of iterations is tricky. If it is too low, we will still be far away from the optimal solution when the algorithm stops. But if it is too high, we will waste time while the model parameters do not change anymore. A simple solution is to set a very large number of iterations but to interrupt the algorithm when the gradient vector becomes small. That is, when its norm becomes smaller than a tiny number $\epsilon$ (called the tolerance). This happens when Gradient Descent has (almost) reached the minimum.


### Dataset

URL: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html#sklearn.datasets.load_boston

This dataset provides housing values in the suburbs of Boston.

The **MEDV** variable is the target variable.

### Data description

The Boston data frame has 506 rows and 14 columns.

This data frame contains the following columns:

- CRIM: per capita crime rate by town.

- ZN: proportion of residential land zoned for lots over 25,000 sq.ft.

- INDUS: proportion of non-retail business acres per town.

- CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

- NOX: nitrogen oxides concentration (parts per 10 million).

- RM: average number of rooms per dwelling.

- AGE: proportion of owner-occupied units built prior to 1940.

- DIS: weighted mean of distances to five Boston employment centres.

- RAD: index of accessibility to radial highways.

- TAX: full-value property-tax rate per $10,000.

- PTRATIO: pupil-teacher ratio by town.

- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

- LSTAT: lower status of the population (percent).

- MEDV: median value of owner-occupied homes in $1000s.

In [1]:
import warnings
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Load Data

First load the data and explore the feature names, target names, etc.

In [2]:
# Load data
boston = datasets.load_boston()
print(boston.data.shape, boston.target.shape)
print(boston.feature_names)

(506, 13) (506,)
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


## Create A DataFrame Object

In [3]:
df = pd.DataFrame(boston.data,columns=boston.feature_names)
df = pd.concat([df,pd.Series(boston.target,name='MEDV')],axis=1)

# Quick Check of the Data

Let’s take a look at the top five rows using the DataFrame’s head() method.


In [4]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


# Description of the Data

DataFrame’s info() method is useful to get a quick description of the data, in particular the total number of rows, and each attribute’s type and number of non-null values.


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
MEDV       506 non-null float64
dtypes: float64(14)
memory usage: 55.4 KB


# Explore the Data: Describe Numerical Attributes

DataFrame's describe() method shows a summary of the numerical attributes.

In [6]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


# Looking for Numerical Correlations with the Target Column

Since the dataset is not too large, we can easily compute the standard correlation coefficient (also called Pearson’s r) between every pair of attributes using DataFrame's corr() method.


In [7]:
# Variable Correlations with the target "MEDV"

df.corr()['MEDV'].sort_values(ascending=False)

MEDV       1.000000
RM         0.695360
ZN         0.360445
B          0.333461
DIS        0.249929
CHAS       0.175260
AGE       -0.376955
RAD       -0.381626
CRIM      -0.388305
NOX       -0.427321
TAX       -0.468536
INDUS     -0.483725
PTRATIO   -0.507787
LSTAT     -0.737663
Name: MEDV, dtype: float64

# Create a Separate Feature Set (Data Matrix X) and Target (1D Vector y)

Create a data matrix (X) that contains all features and a 1D target vector (y) containing the target.



In [8]:
# It contains all attributes (all features and the target)
allData = df


# Use the following code to select ALL features
X = df.drop(columns='MEDV')  # Data Matrix containing all features excluding the target


y = df['MEDV'] # 1D targer vector

print(X.shape)
print(y.shape)

(506, 13)
(506,)


# Create Train and Test Dataset

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale The Features

We should ensure that all features have a similar scale. Otherwise optimization algorithms (e.g., Gradient Descent based algorithms) will take much longer time to converge.

Also, regularization techniques are sensitive to the scale of data. Thus, we must scale the features before applying regularization.

In [10]:
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(X_train)

# Apply transform to both the training set and the test set.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Implementation of Sklearn's Stochastic Gradient Descent (SGD)


Sklearn provides a SGDRegressor model. We will implement the regularized SGDRegressor.


### Regularization

Regularization is an effective technique for reducing overfitting.

For a linear model, regularization is typically achieved by constraining the weights of the model. We will now look at three different ways to constrain the weights.

- Ridge Regression ($l_2$ norm)
- Lasso Regression ($l_1$ norm)
- Elastic Net (it combines $l_1$ and $l_2$ priors as regularizer)


### Regularization: A Common Hyperparameter

In all regularization methods, we need to use the hyperparameter $\alpha$. It controls how much we want to regularize the model. 

- If $\alpha = 0$ then Ridge Regression is just Linear Regression. 
- If $\alpha$ is very large, then all weights end up very close to zero and the result is a flat line going through the data’s mean. 

## Evaluation Metrics

We will use two evaluation metrics.

- Mean Squared Error (MSE)
- Coefficient of Determination or $R^2$/$r^2$


### Note on $R^2$:
R-squared is a statistical measure of how close the data are to the fitted regression line. 

R-squared measures the proportion of the variance in the dependent variable that is predictable from the independent variable(s).

R-squared = Explained variation / Total variation

R-squared is always between 0 and 100%:

- 0% indicates that the model explains none of the variability of the response data around its mean.
- 100% indicates that the model explains all the variability of the response data around its mean.

In general, the higher the R-squared, the better the model fits your data. 

#### Compute $R^2$ using the sklearn:

- The "score" function of the OLS Linear Regression object
- The "r2_score" function from sklearn.metrics

#### Compute MSE using the sklearn:

- The "mean_squared_error" function from sklearn.metrics


## SGD Regressor



In the Stochastic Gradient Descent algorithm the gradient of the loss is estimated each sample at a time and the model is updated along the way with a decreasing strength schedule (aka learning rate).

The regularizer is a penalty added to the loss function that shrinks model parameters towards the zero vector using either the squared euclidean norm $l_2$ or the absolute norm $l_1$ or a combination of both (Elastic Net). If the parameter update crosses the 0.0 value because of the regularizer, the update is truncated to 0.0 to allow for learning sparse models and achieve online feature selection.


We need to set the following hyperparameters.


- penalty : str, ‘none’, ‘l2’, ‘l1’, or ‘elasticnet’

The penalty (aka regularization term) to be used. Defaults to ‘l2’ which is the standard regularizer for linear SVM models. ‘l1’ and ‘elasticnet’ might bring sparsity to the model (feature selection) not achievable with ‘l2’.

- alpha : float

Constant that multiplies the regularization term. Defaults to 0.0001 Also used to compute learning_rate when set to ‘optimal’.

- l1_ratio : float

The Elastic Net mixing parameter, with 0 <= l1_ratio <= 1. l1_ratio=0 corresponds to L2 penalty, l1_ratio=1 to L1. Defaults to 0.15.

- learning_rate : string, optional

The learning rate schedule:

-- ‘constant’:
eta = eta0

-- ‘optimal’:
eta = 1.0 / (alpha * (t + t0)) where t0 is chosen by a heuristic proposed by Leon Bottou.

-- ‘invscaling’: [default]
eta = eta0 / pow(t, power_t)

- eta0 : double

The initial learning rate for the ‘constant’ or ‘invscaling’ schedules. The default value is 0.0 as eta0 is not used by the default schedule ‘optimal’.


- max_iter : int, optional

The maximum number of passes over the training data (aka epochs). It only impacts the behavior in the fit method, and not the partial_fit. Defaults to 5. 


- tol : float or None, optional

The stopping criterion. If it is not None, the iterations will stop when (loss > previous_loss - tol). Defaults to None. Defaults to 1e-3 from 0.21.



## Task 1: Implement SGD on a Linear Regression Model

First we implement a SGD Linear Regression model without hyperparameter tuning. We do this first just to get a sense of the model's performance. Then, we perform hyperparameter tuning to determine optimal values for the hyperparameters.

In [11]:
# SGD Regression

# Create SGDRegressor linear regression object
lin_reg_sgd = SGDRegressor(max_iter=1000, eta0=0.01, penalty="elasticnet", l1_ratio=0.0, alpha=0.01)

# Train the model
lin_reg_sgd.fit(X_train, y_train)



# The intercept
print("Intercept: \n", lin_reg_sgd.intercept_)

# The coefficients
print("Coefficients: \n", lin_reg_sgd.coef_)

# The number of iterations
print("Number of Iterations: \n", lin_reg_sgd.n_iter_)


print("\n----------------------------- Model Evaluation -----------------------------")

# Make prediction 
y_train_predicted_sgd = lin_reg_sgd.predict(X_train)
y_test_predicted_sgd = lin_reg_sgd.predict(X_test)


print("Training: Mean squared error: %.2f"
      % mean_squared_error(y_train, y_train_predicted_sgd))

print("Test: Mean squared error: %.2f"
      % mean_squared_error(y_test, y_test_predicted_sgd))


# Explained variance score: 1 is perfect prediction
print("\nTraining: Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % 
      r2_score(y_train, y_train_predicted_sgd))




# Explained variance score: 1 is perfect prediction
print("Test: Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % 
      r2_score(y_test, y_test_predicted_sgd))

Intercept: 
 [22.78652701]
Coefficients: 
 [-0.92241108  0.51371146  0.01958236  0.72903868 -1.75432889  3.19350245
 -0.17296238 -2.81189611  1.45094904 -1.01407278 -1.97814575  1.12098063
 -3.52327863]
Number of Iterations: 
 30

----------------------------- Model Evaluation -----------------------------
Training: Mean squared error: 21.76
Test: Mean squared error: 24.77

Training: Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.75
Test: Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.66


## Model Selection: Hyperparameter Tunining


A regression model is defined by a set of parameters: alpha, l1_ratio, etc. These are called hyperparameters.

We need to select the best model based on the optimal values of these hyperparameters. This process is called hyperparameter tuning.

The best way to do hyperparameter tuning is to use **cross-validation**.

We will use Scikit-Learn’s GridSearchCV to search the combinations of hyperparameter values that provide best performance.

We need to tell which hyperparameters we want the GridSearchCV to experiment with, and what values to try out. It will evaluate all the possible combinations of hyperparameter values, using cross-validation. 


### Important:

The GridSearchCV takes an argument to define the scoring metric (performance measure). 

See the list of possible scoring functions:
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

For regression, we may use "neg_mean_squared_error" or "explained_variance" scoring function. 



## Linear Regression: Hyperparameter Tuning for SGD Regressor

In [12]:
%%time

warnings.filterwarnings('ignore')

# The param_grid tells Scikit-Learn to evaluate all combinations of the hyperparameter values
param_grid = {'alpha': [0.1, 0.01, 0.001], 'learning_rate': ["constant", "optimal", "invscaling"], 
              'l1_ratio': [1, 0.5, 0.2, 0], 'max_iter':[100, 400, 1000, 10000],'eta0': [0.01, 0.001],
              'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']}



sgd = SGDRegressor()

sgd_cv = GridSearchCV(sgd, param_grid, scoring='neg_mean_squared_error', cv=10, verbose=2, n_jobs=-1)
sgd_cv.fit(X_train, y_train)


params_optimal_sgd = sgd_cv.best_params_

print("Best Score (negative mean squared error): %f" % sgd_cv.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal_sgd)
print("\n")

Fitting 10 folds for each of 1152 candidates, totalling 11520 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 1452 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 3124 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 5388 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 9468 tasks      | elapsed:   33.4s


Best Score (negative mean squared error): -23.776077
Optimal Hyperparameter Values:  {'alpha': 0.01, 'eta0': 0.01, 'l1_ratio': 0.5, 'learning_rate': 'optimal', 'loss': 'squared_loss', 'max_iter': 10000}


CPU times: user 11.2 s, sys: 358 ms, total: 11.6 s
Wall time: 38.7 s


[Parallel(n_jobs=-1)]: Done 11520 out of 11520 | elapsed:   38.6s finished


## Linear Regression: Select The Best Model for the SGD Regressor

Using the optimal hyperparameter values, create the best model.
Then, fit the model.



In [13]:
# SGD Regression

# Create SGDRegressor linear regression object using the optimal hyperparameter values
lin_reg_sgd = SGDRegressor(**params_optimal_sgd)

# Train the model
lin_reg_sgd.fit(X_train, y_train)


# The intercept
print("Intercept: \n", lin_reg_sgd.intercept_)

# The coefficients
print("Coefficients: \n", lin_reg_sgd.coef_)

# The number of iterations
print("Number of Iterations: \n", lin_reg_sgd.n_iter_)


print("\n----------------------------- Model Evaluation -----------------------------")

# Make prediction 
y_train_predicted_sgd = lin_reg_sgd.predict(X_train)


print("Mean squared error: %.2f"
      % mean_squared_error(y_train, y_train_predicted_sgd))


# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y_train, y_train_predicted_sgd))

Intercept: 
 [22.84544931]
Coefficients: 
 [-0.9022      0.52711153  0.12695643  0.69354744 -1.83666141  3.16125689
 -0.10066126 -3.03521397  1.77972575 -1.1865263  -1.97955591  1.10732345
 -3.50102086]
Number of Iterations: 
 95

----------------------------- Model Evaluation -----------------------------
Mean squared error: 21.89
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.75


## Linear Regression: Evaluate Model Performance for SGD Regressor

Evaluate the model's performance using cross-validation. 

Use Scikit-Learn's cross_val_score function. 

Note that the "scoring" argument should be set based on the type of classification (binary/multiclass).

In [14]:
# Scoring Parameter for Regression:
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

scores = cross_val_score(lin_reg_sgd, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
print(scores)

print("Negative Mean Squared Error: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[-14.74784841 -17.93447856 -28.46053203 -46.01235169 -21.53966764
 -26.48782046 -18.83904573 -20.53088499 -14.94533691 -32.80433796]
Negative Mean Squared Error: -24.23 (+/- 18.29)


## Linear Regression: Evaluate Model Performance Using Test Data

In [15]:
# Make prediction using the test data
y_test_predicted = lin_reg_sgd.predict(X_test)


test_mse_linear = mean_squared_error(y_test, y_test_predicted)

print("Mean squared error: %.2f"
      % test_mse_linear)


# Explained variance score: 1 is perfect prediction
test_r2_linear = r2_score(y_test, y_test_predicted)
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" 
      % test_r2_linear)

Mean squared error: 24.47
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.67


## Task 2: Implement SGD on a Polynomial Regression Model

We will use optimal polynomial degree obtained from previous notebook on this dataset.

In [16]:
# Variable that speficies the degree of the polynomial to be added to the feature vector
poly_degree = 2


# Add polynomial terms with the feature vector using the sklearn PolynomialFeatures class
poly_features = PolynomialFeatures(degree=poly_degree, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train)


print("No. of Original Features: ", X_train.shape[1])
print("No. of Augmented Features: ", X_train_poly.shape[1])

No. of Original Features:  13
No. of Augmented Features:  104


## Polynomial Regression: Hyperparameter Tuning for SGD Regressor

In [17]:
%%time

warnings.filterwarnings('ignore')

# The param_grid tells Scikit-Learn to evaluate all combinations of the hyperparameter values
# param_grid = {'alpha': [0.1, 0.01], 'learning_rate': ["invscaling"], 
#               'l1_ratio': [1, 0.5, 0.2, 0], 'max_iter':[100, 400, 1000, 5000],'eta0': [0.01, 0.001, 0.0001]}


param_grid = {'alpha': [0.1, 0.01], 'learning_rate': ["invscaling"], 
              'l1_ratio': [1, 0.5, 0.2, 0], 'max_iter':[100, 400, 1000, 10000],'eta0': [0.01, 0.001, 0.0001],
              'loss': ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']}



sgd = SGDRegressor()

sgd_cv = GridSearchCV(sgd, param_grid, scoring='neg_mean_squared_error', cv=10, verbose=2, n_jobs=-1)
sgd_cv.fit(X_train_poly, y_train)


params_optimal_sgd = sgd_cv.best_params_

print("Best Score (negative mean squared error): %f" % sgd_cv.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal_sgd)
print("\n")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 10 folds for each of 384 candidates, totalling 3840 fits


[Parallel(n_jobs=-1)]: Done 214 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 1400 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 1926 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-1)]: Done 3606 tasks      | elapsed:   48.0s


Best Score (negative mean squared error): -14.951719
Optimal Hyperparameter Values:  {'alpha': 0.1, 'eta0': 0.001, 'l1_ratio': 0, 'learning_rate': 'invscaling', 'loss': 'squared_epsilon_insensitive', 'max_iter': 10000}


CPU times: user 5.42 s, sys: 489 ms, total: 5.9 s
Wall time: 55.1 s


[Parallel(n_jobs=-1)]: Done 3840 out of 3840 | elapsed:   55.1s finished


## Polynomial Regression: Select The Best Model for the SGD Regressor

Using the optimal hyperparameter values, create the best model.
Then, fit the model.



In [18]:
# SGD Regression

# Create SGDRegressor linear regression object using the optimal hyperparameter values
lin_reg_sgd = SGDRegressor(**params_optimal_sgd)

# Train the model
lin_reg_sgd.fit(X_train_poly, y_train)


# # The intercept
# print("Intercept: \n", lin_reg_sgd.intercept_)

# # The coefficients
# print("Coefficients: \n", lin_reg_sgd.coef_)

# The number of iterations
print("Number of Iterations: \n", lin_reg_sgd.n_iter_)


print("\n----------------------------- Model Evaluation -----------------------------")

# Make prediction 
y_train_predicted_sgd = lin_reg_sgd.predict(X_train_poly)


print("Mean squared error: %.2f"
      % mean_squared_error(y_train, y_train_predicted_sgd))


# Explained variance score: 1 is perfect prediction
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y_train, y_train_predicted_sgd))

Number of Iterations: 
 401

----------------------------- Model Evaluation -----------------------------
Mean squared error: 8.02
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.91


## Polynomial Regression: Evaluate Model Performance Using Test Data

In [19]:
poly_features = PolynomialFeatures(degree=poly_degree, include_bias=False)
X_test_poly = poly_features.fit_transform(X_test)

# Make prediction using the test data
y_test_predicted = lin_reg_sgd.predict(X_test_poly)

test_mse_polynomial = mean_squared_error(y_test, y_test_predicted)

print("Mean squared error: %.2f"
      % test_mse_polynomial)



# Explained variance score: 1 is perfect prediction

test_r2_polynomial = r2_score(y_test, y_test_predicted)
print("Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" 
      % test_r2_polynomial)

Mean squared error: 14.33
Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.80


## SGD Linear Regression vs. SGD Polynomial Regression 

We observe that the SGD Polynomial Regression performs significantly better. It uses regularization to reduce the overfitting problem.

In [20]:
data = [["MSE (test)", test_mse_linear, test_mse_polynomial], 
        ["R2 Score (test)", test_r2_linear, test_r2_polynomial]]
pd.DataFrame(data, columns=["Metric", "SGD Linear Regression", "SGD Polynomial Regression (degree 2)"])

,Metric,SGD Linear Regression,SGD Polynomial Regression (degree 2)
0,MSE (test),24.474214,14.32867
1,R2 Score (test),0.666263,0.80461


# Beyond Linear Regression

Based on all experiments that we have done so far, we observe that even the optimal Linear Regression model (**regularized polynomial regression of degree 2**, implemented in notebook 3) is unable to reduce the MSE below 12. 

To further reduce the MSE we will have to use more sophisticated regression models. Below we apply two advanced regressors on the same dataset:
- Support Vector Machine (Gaussian Radial Basis Function)
- Random Forest

We did not fine tune the hyperparameters. Please refer to the notebooks on these two models for a detail discussion on their hyperparameters. Below we use empirically tested optimal values for the hyperparameters. Our goal is to illustrate the superiority of these advanced models.


## Support Vector Machine (Gaussian Radial Basis Function)

In [21]:
from sklearn.svm import SVR


svm = SVR(kernel='rbf', C=1000.0, gamma=0.01)
svm.fit(X_train, y_train)


print("\n----------------------------- Model Evaluation -----------------------------")

# Make prediction 
y_train_predicted_svm = svm.predict(X_train)


print("Train: Mean squared error: %.2f"
      % mean_squared_error(y_train, y_train_predicted_svm))


# Explained variance score: 1 is perfect prediction
print("Train: Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" % r2_score(y_train, y_train_predicted_svm))


# Make prediction using the test data
y_test_predicted_svm = svm.predict(X_test)

test_mse_svm = mean_squared_error(y_test, y_test_predicted_svm)

print("\nTest: Mean squared error: %.2f"
      % test_mse_svm)



# Explained variance score: 1 is perfect prediction

test_r2_svm = r2_score(y_test, y_test_predicted_svm)
print("Test: Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" 
      % test_r2_svm)


----------------------------- Model Evaluation -----------------------------
Train: Mean squared error: 5.71
Train: Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.93

Test: Mean squared error: 10.63
Test: Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.86


## Random Forest

To use the Random Forest model we need the unscaled features. Thus we read the features from the DataFrame object first.

In [22]:
# Use the following code to select ALL features
X = allData.drop(columns='MEDV')  # Data Matrix containing all features excluding the target


y = allData['MEDV'] # 1D targer vector

print(X.shape)
print(y.shape)


# Create train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(506, 13)
(506,)


In [25]:
from sklearn.ensemble import RandomForestRegressor


rnd_forest_reg = RandomForestRegressor(n_estimators=500, criterion="mse", max_features="auto", 
                                       verbose=1, max_depth=8, 
                                       oob_score=True, n_jobs=-1)

rnd_forest_reg.fit(X_train, y_train)


# Make prediction 
y_train_predicted_rnd_forest = rnd_forest_reg.predict(X_train)


train_mse_rnd_forest = mean_squared_error(y_train, y_train_predicted_rnd_forest)

print("Train: Mean squared error: %.2f"
      % train_mse_rnd_forest)


# Explained variance score: 1 is perfect prediction
print("Train: Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" 
      % r2_score(y_train, y_train_predicted_rnd_forest))




y_test_predicted_rnd_forest = rnd_forest_reg.predict(X_test)


test_mse_rnd_forest = mean_squared_error(y_test, y_test_predicted_rnd_forest)

print("Test: Mean squared error: %.2f"
      % test_mse_rnd_forest)


# Explained variance score: 1 is perfect prediction

test_r2_rnd_forest = r2_score(y_test, y_test_predicted_rnd_forest)


print("Test: Coefficient of determination r^2 variance score [1 is perfect prediction]: %.2f" 
      % test_r2_rnd_forest)

#print("\nScore of the training dataset obtained using an out-of-bag estimate: ", rnd_forest_reg.oob_score_)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.5s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s


Train: Mean squared error: 2.75
Train: Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.97
Test: Mean squared error: 8.48
Test: Coefficient of determination r^2 variance score [1 is perfect prediction]: 0.88


[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.1s finished


# Comparison of Linear Regression & Advanced Regressor Models

In [26]:
data = [
        ["SGD Linear Regression", test_mse_linear, test_r2_linear], 
        ["SGD Polynomial Regression (degree 2)", test_mse_polynomial, test_r2_polynomial],
        ["Support Vector Machine (Gaussian RBF)", test_mse_svm, test_r2_svm],
        ["Random Forest", test_mse_rnd_forest, test_r2_rnd_forest]
       ]


pd.DataFrame(data, columns=["Model", "MSE (test)", "R2 Score (test)"])

,Model,MSE (test),R2 Score (test)
0,SGD Linear Regression,24.474214,0.666263
1,SGD Polynomial Regression (degree 2),14.328670,0.804610
2,Support Vector Machine (Gaussian RBF),10.627752,0.855077
3,Random Forest,8.482888,0.884325


# Final Observation

We observe that Random Forest performs significantly better than Linear Regression reducing the MSE below 9.